In [1]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

In [30]:
# Importing the csv

data = pd.read_csv('D:\\Python\\Project\\Phonepe\\phonepe\\files\\cleaned\\district_user_C.csv') 
df = pd.DataFrame(data)

In [31]:
df.head()

,Unnamed: 0,State,Year,Quater,District,Registered_user,App_opening,id
0,0,Andaman & Nicobar Island,2018,Q1,North And Middle Andaman District,632,0,35
1,1,Andaman & Nicobar Island,2018,Q1,South Andaman District,5846,0,35
2,2,Andaman & Nicobar Island,2018,Q1,Nicobars District,262,0,35
3,3,Andaman & Nicobar Island,2018,Q2,North And Middle Andaman District,911,0,35
4,4,Andaman & Nicobar Island,2018,Q2,South Andaman District,8143,0,35


In [32]:
df.drop(columns=["Unnamed: 0"], inplace=True)

In [27]:
df.rename(columns={'District&pincode.1' : 'District_pincode'}, inplace=True)

In [33]:
df.columns

Index(['State', 'Year', 'Quater', 'District', 'Registered_user', 'App_opening',
       'id'],
      dtype='object')

In [36]:
try:
    connection = mysql.connector.connect(host='localhost',
                                            database='Phonepe',
                                            user='root',
                                            password='Root@0000')
    if connection.is_connected():
        cursor = connection.cursor()
        cursor.execute('DROP TABLE IF EXISTS district_user_C;')
        print('Creating table....')

        query = '''CREATE TABLE district_user_C (State nvarchar (50),
                                            Year int,
                                            Quater nvarchar(3), 
                                            District nvarchar(50),
                                            Registered_user bigint,
                                            App_opening bigint(50),
                                            id int) '''

        cursor.execute(query)
        print("district_user_C table is created....")

        for i,row in df.iterrows():
            sql = "INSERT INTO district_user_C VALUES (%s,%s,%s,%s,%s,%s,%s)"
            cursor.execute(sql, tuple(row))
            
            # the connection is not autocommitted by default, so we must commit to save our changes
            connection.commit()
        print("Record inserted")
except Error as e:
    print("Error while connecting to MySQL", e)

Creating table....
district_user_C table is created....
Record inserted


In [6]:
def import_files():
    data_frame = ['district_user_C', 'state_transaction_C', 'top_transaction_C', 'top_users_C']
    imported_files = []
    for name in data_frame:
        try:
            connection = mysql.connector.connect(host='localhost',
                                                    database='Phonepe',
                                                    user='root',
                                                    password='Root@0000')
            if connection.is_connected():
                cursor = connection.cursor()

                cursor.execute(f"SELECT * FROM {name}")
                records1 = cursor.fetchall()
                name = pd.DataFrame(records1,
                                columns=[i[0] for i  in cursor.description])
                
                imported_files.append(name)

                connection.commit()
                cursor.close()
                connection.close()
        except Error as e:
            print("Table Not found: ", e)
    print("Import completed")

    return imported_files

In [7]:
imported_files = import_files()

Import completed


In [8]:
district_user = imported_files[0]
state_transaction = imported_files[1]
top_transaction = imported_files[2]
top_users = imported_files[3]

In [9]:
district_user

,State,Year,Quater,District,Registered_user,App_opening,id
0,Andaman & Nicobar Island,2018,Q1,North And Middle Andaman District,632,0,35
1,Andaman & Nicobar Island,2018,Q1,South Andaman District,5846,0,35
2,Andaman & Nicobar Island,2018,Q1,Nicobars District,262,0,35
3,Andaman & Nicobar Island,2018,Q2,North And Middle Andaman District,911,0,35
4,Andaman & Nicobar Island,2018,Q2,South Andaman District,8143,0,35
...,...,...,...,...,...,...,...
14635,West Bengal,2022,Q4,Nadia District,1359420,33853990,19
14636,West Bengal,2022,Q4,Birbhum District,855236,20950662,19
14637,West Bengal,2022,Q4,Purba Medinipur District,1346908,38278506,19
14638,West Bengal,2022,Q4,Maldah District,954892,29023743,19
